In [1]:
# Okay time for our K-Means algorithm! woop woop!

# We want our Kmeans algorithm not only to cluster by k given cluster, it should also be able to find best k
# this will be done by an approach from the distortion theory, but later more on that. For now we just need to keep
# that in mind for our approach tactic

# 1. We need to take k+1 (starting with k=1) data points with k_max =< amount of values. (k_1+1,...,k_max) 

# 2. Calculate the euclidean distance to all data points and asign each to the class of the closest datapoint
# 3. Calculate each center of mass for this distribution
# 4. Repeat 2 and 3 m times until the centers of mass don't change and then continue with 1

# 5. Now just insert the results in the formula to best k by finding the lowest rate of distortion withing the calculated clusters

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from math import sqrt
from sklearn import datasets
from sklearn.datasets import make_blobs

# Create a clustered dataset to use
ds,y = make_blobs(n_samples=800, n_features=3, centers=4)

# Just to have it also as a dataframe, maybe we want to work with this
df = pd.DataFrame (ds, y)
df['cluster'] = y

In [3]:
# 1. Get random starting points.
# when the logic on the simple model works, we will just embedd an incrementer on k and then compare the results for different k's
# only important thing till now is, k has to be changable and the code stil needs to work

clusters = []
centroids = {}
k = 2
i=0
l = 0


sample_numbers = random.sample(range(1, len(df)), k)

while i < k:
    centroids[i] = ds[sample_numbers[i]]
    clusters.append(list(centroids[i]))
    i = i + 1                 #else: call next function
    
# fine now we have a list of centroids in our list of clusters
clusters

[[-7.178472473140733, 8.386998659799792, 3.7006179542592066],
 [4.0404663732848105, 5.0569178666249135, 5.122295915461905]]

In [7]:
def euclidean_distance_on_array(target, distance_to):
    distance = 0.0
    for n in range(0,k-1): 
        distance = distance + (target[n] - distance_to[n])**2
    return sqrt(distance)

euclidean_distance_on_array(centroids[0], ds[42])



[-7.178472473140733, 8.386998659799792, 3.7006179542592066]
[4.0404663732848105, 5.0569178666249135, 5.122295915461905]


In [12]:
def clustering(clusterspoints, dataset):
    j = 0
    calc = {}
    compare=[]
    for row in dataset:
        for clpo in clusterspoints:
            calc[j] = euclidean_distance_on_array(row, clpo)
            compare.append(calc[j])
    j = j+1
    return (calc)
        
            
            
###
        


clustered = clustering(clusters, ds)
clustered

{0: 0.7285816232705038}

In [ ]:
e = 0
c = 0
for clusterpoint in clusters:
    
    for row in ds:
        euclidean_distance_on_array(row, clusters[j])
    

In [ ]:
            if euclidean_distance_on_array(row, clusters[j]) <= euclidean_distance_on_array(row,clusters[j+1]):
                clusters[j].append(euclidean_distance_on_array(row,clusters[j]))
                j = j+1
            else:
                j = j+1
                clusters[j].append(euclidean_distance_on_array(row,centroids[j]))

In [ ]:
centers = {}
for c in range(0,k):
    centers[c] = centroids[c]
    c = c+1